In [1]:
import pandas as pd
import numpy as np

In [2]:
df1_1 = pd.read_excel('/content/drive/MyDrive/国赛题/附件1：123家有信贷记录企业的相关数据.xlsx',sheet_name=0)
df1_2 = pd.read_excel('/content/drive/MyDrive/国赛题/附件1：123家有信贷记录企业的相关数据.xlsx',sheet_name=1)
df1_3 = pd.read_excel('/content/drive/MyDrive/国赛题/附件1：123家有信贷记录企业的相关数据.xlsx',sheet_name=2)

df2_1 = pd.read_excel('/content/drive/MyDrive/国赛题/附件2：302家无信贷记录企业的相关数据.xlsx',sheet_name=0)
df2_2 = pd.read_excel('/content/drive/MyDrive/国赛题/附件2：302家无信贷记录企业的相关数据.xlsx',sheet_name=1)
df2_3 = pd.read_excel('/content/drive/MyDrive/国赛题/附件2：302家无信贷记录企业的相关数据.xlsx',sheet_name=2)

df3_1 = pd.read_excel('/content/drive/MyDrive/国赛题/附件3：银行贷款年利率与客户流失率关系的统计数据.xlsx')

##### 有这三个影响公司能否还贷的指标是可以利用现有数据构造出来的，成立年限，客户数 ，年均收益率。 


In [3]:
df1_2.head()

,企业代号,发票号码,开票日期,销方单位代号,金额,税额,价税合计,发票状态
0,E1,3390939,2017-07-18,A00297,-943.40,-56.60,-1000.00,有效发票
1,E1,3390940,2017-07-18,A00297,-4780.24,-286.81,-5067.05,有效发票
2,E1,3390941,2017-07-18,A00297,943.40,56.60,1000.00,有效发票
3,E1,3390942,2017-07-18,A00297,4780.24,286.81,5067.05,有效发票
4,E1,9902669,2017-08-07,A05061,326.21,9.79,336.00,有效发票


In [4]:
#每家公司成立年限，假设为第一张发票时间到最后一张发票时间的差值，以月为单位
#销项发票
live_time2 = df1_2[['企业代号','开票日期']]
live_time2_min = live_time2.groupby('企业代号').min().reset_index()
live_time2_max = live_time2.groupby('企业代号').max().reset_index()
live_time2 = pd.merge(live_time2_max,live_time2_min,on='企业代号')
live_time2['生存年限1'] = live_time2['开票日期_x'] - live_time2['开票日期_y']

#进项发票
live_time3 = df1_3[['企业代号','开票日期']]
live_time3_min = live_time3.groupby('企业代号').min().reset_index()
live_time3_max = live_time3.groupby('企业代号').max().reset_index()
live_time3 = pd.merge(live_time3_max,live_time3_min,on='企业代号')
live_time3['生存年限2'] = live_time3['开票日期_x'] - live_time3['开票日期_y']

#找出生存年限最大值作为公司的生存年限 
live_time = pd.merge(live_time2,live_time3,on='企业代号')[['企业代号','生存年限1','生存年限2']]
x1 = list(live_time['生存年限1'])
x2 = list(live_time['生存年限2'])
x = []
for i in range(len(x1)):
  if x1[i]>x2[i]:
    x.append(x1[i])
  else:
    x.append(x2[i])
company_id = pd.DataFrame(live_time['企业代号'])
x = pd.DataFrame(x,columns=['生存年限'])
live_time = company_id.join(x)

In [5]:
live_time

,企业代号,生存年限
0,E1,941 days
1,E10,1116 days
2,E100,1110 days
3,E101,452 days
4,E102,1141 days
...,...,...
118,E95,1111 days
119,E96,702 days
120,E97,564 days
121,E98,1091 days


In [6]:
#划分开票日期的每个月
live_time4 = df1_3[['企业代号','开票日期']]
live_time4['开票日期'] = live_time4['开票日期'].astype(str)
live_time4['开票日期'] = live_time4['开票日期'].str.slice(0,7)
df1_3['开票日期'] = live_time4['开票日期'] 

live_time5 = df1_2[['企业代号','开票日期']]
live_time5['开票日期'] = live_time5['开票日期'].astype(str)
live_time5['开票日期'] = live_time5['开票日期'].str.slice(0,7)
df1_2['开票日期'] = live_time5['开票日期'] 

live_time6 = df2_2[['企业代号','开票日期']]
live_time6['开票日期'] = live_time6['开票日期'].astype(str)
live_time6['开票日期'] = live_time6['开票日期'].str.slice(0,7)
df2_2['开票日期'] = live_time6['开票日期'] 

live_time7 = df2_3[['企业代号','开票日期']]
live_time7['开票日期'] = live_time7['开票日期'].astype(str)
live_time7['开票日期'] = live_time7['开票日期'].str.slice(0,7)
df2_3['开票日期'] = live_time7['开票日期']


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

NameError: ignored

In [10]:
live_time['生存年限'] = live_time['生存年限'].astype(str)
live_time['生存年限'] = live_time['生存年限'].str.split().str.get(0)
live_time['生存年限'] = live_time['生存年限'].map(lambda x: str(int(x)//30))

In [11]:
df1_3.head()

,企业代号,发票号码,开票日期,购方单位代号,金额,税额,价税合计,发票状态
0,E1,11459356,2017-08,B03711,9401.71,1598.29,11000.0,有效发票
1,E1,5076239,2017-08,B00844,8170.94,1389.06,9560.0,有效发票
2,E1,5076240,2017-08,B00844,8170.94,1389.06,9560.0,有效发票
3,E1,5076241,2017-08,B00844,4085.47,694.53,4780.0,有效发票
4,E1,5076242,2017-08,B00844,4085.47,694.53,4780.0,有效发票


In [12]:
#选择客户数作为第二个指标。以有效进项收入发票作为依据
down_client = df1_3[['企业代号','购方单位代号','发票状态','开票日期']]
down_client = down_client[down_client['发票状态']=='有效发票']
down_client = down_client.drop_duplicates(subset=['企业代号','购方单位代号'])
down_client = down_client.groupby(['企业代号','开票日期'])['购方单位代号'].count().reset_index()
down_client.rename(columns={'购方单位代号':'下游企业数'},inplace = True)
down_client


,企业代号,开票日期,下游企业数
0,E1,2017-08,106
1,E1,2017-09,16
2,E1,2017-10,20
3,E1,2017-11,12
4,E1,2017-12,14
...,...,...,...
2289,E98,2019-06,1
2290,E98,2019-11,1
2291,E98,2019-12,2
2292,E99,2018-06,1


In [13]:
#选择客户数作为第二个指标。以有效进项收入发票作为依据
up_client = df1_2[['企业代号','销方单位代号','发票状态','开票日期']]
up_client = up_client[up_client['发票状态']=='有效发票']
up_client = up_client.drop_duplicates(subset=['企业代号','销方单位代号'])
up_client = up_client.groupby(['企业代号','开票日期'])['销方单位代号'].count().reset_index()
up_client.rename(columns={'销方单位代号':'上游游企业数'},inplace = True)
up_client


,企业代号,开票日期,上游游企业数
0,E1,2017-07,1
1,E1,2017-08,17
2,E1,2017-09,33
3,E1,2017-10,21
4,E1,2017-11,24
...,...,...,...
2918,E99,2019-07,1
2919,E99,2019-08,1
2920,E99,2019-09,1
2921,E99,2019-10,5


In [34]:
#年收益率通过有效发票"销项金额"和"进项金额"的均值和标准差
#销项
outcome_money = df1_2[['企业代号','金额','开票日期']]
outcome_money_mean = outcome_money.groupby(['企业代号','开票日期'])['金额'].mean().reset_index()
#outcome_money_std = outcome_money.groupby(['企业代号','开票日期'])['金额'].agg(np.std).reset_index()
#进项
in_money = df1_3[['企业代号','金额','开票日期']]
in_money_mean = in_money.groupby(['企业代号','开票日期'])['金额'].mean().reset_index()
#in_money_std = outcome_money.groupby(['企业代号','开票日期'])['金额'].agg(np.std).reset_index()
#进项销项concat
out_in_mean = pd.merge(outcome_money_mean,in_money_mean,on=['企业代号','开票日期'])
out_in_mean.rename(columns={'金额_x':'销项每月均值',
                           '金额_y':'进项每月均值',
                           },inplace=True)
out_in_mean

,企业代号,开票日期,销项每月均值,进项每月均值
0,E1,2017-08,3.907968e+06,584348.180208
1,E1,2017-09,2.589242e+06,680427.107283
2,E1,2017-10,2.623279e+06,471804.535748
3,E1,2017-11,3.110492e+06,655526.807781
4,E1,2017-12,1.802145e+06,421278.150859
...,...,...,...,...
2811,E99,2018-06,2.999114e+02,222527.630000
2812,E99,2018-08,4.482776e+03,423492.640000
2813,E99,2018-12,6.773801e+03,8.620000
2814,E99,2019-01,1.092904e+03,104753.996667


In [33]:
#年收益率通过有效发票"销项金额"和"进项金额"的均值和标准差
#销项
outcome_money = df1_2[['企业代号','金额','开票日期']]

outcome_money_std = outcome_money.groupby(['企业代号','开票日期'])['金额'].agg(np.std).reset_index()
#进项
in_money = df1_3[['企业代号','金额','开票日期']]

in_money_std = in_money.groupby(['企业代号','开票日期'])['金额'].agg(np.std).reset_index()
#进项销项concat
out_in_std = pd.merge(outcome_money_std,in_money_std,on=['企业代号','开票日期'])
out_in_std.rename(columns={'金额_x':'销项每月金额标准差',
                           '金额_y':'进项每月金额标准差',
                           },inplace=True)
out_in_std

,企业代号,开票日期,销项每月金额标准差,进项每月金额标准差
0,E1,2017-08,6.288336e+06,401258.871272
1,E1,2017-09,5.340444e+06,434914.064999
2,E1,2017-10,5.609322e+06,501831.882727
3,E1,2017-11,5.892223e+06,394124.045268
4,E1,2017-12,4.420393e+06,350723.314608
...,...,...,...,...
2811,E99,2018-06,1.041958e+02,0.000000
2812,E99,2018-08,1.718330e+04,NaN
2813,E99,2018-12,2.070157e+04,NaN
2814,E99,2019-01,3.074831e+03,35400.583939


In [37]:
#年收益率通过有效发票"销项税合计"和"进项税合计"
#销项
outcome_sum = df1_2[['企业代号','价税合计','开票日期']]
outcome_sum = outcome_sum.groupby(['企业代号','开票日期'])['价税合计'].sum().reset_index()
#进项
in_sum = df1_3[['企业代号','价税合计','开票日期']]
in_sum = in_sum.groupby(['企业代号','开票日期'])['价税合计'].sum().reset_index()
#进项销项concat
out_in_sum = pd.merge(outcome_sum,in_sum,on=['企业代号','开票日期'])
out_in_sum.rename(columns={'价税合计_x':'销项税合计',
                           '价税合计_y':'进项税合计',
                           },inplace=True)
out_in_sum

,企业代号,开票日期,销项税合计,进项税合计
0,E1,2017-08,2.789028e+08,1.975857e+08
1,E1,2017-09,2.968568e+08,2.842076e+08
2,E1,2017-10,1.994778e+08,1.661554e+08
3,E1,2017-11,3.020342e+08,2.316238e+08
4,E1,2017-12,1.938629e+08,9.759330e+07
...,...,...,...,...
2811,E99,2018-06,4.870660e+03,4.584069e+05
2812,E99,2018-08,1.820008e+05,4.361974e+05
2813,E99,2018-12,5.001701e+05,9.480000e+00
2814,E99,2019-01,5.937568e+04,3.236898e+05


In [15]:
#计算每个企业进项，作废发票数，有效发票数
bill_income = df1_2[['企业代号','价税合计','发票状态','开票日期']]

#有效发票数
valid_bill_income = bill_income[bill_income['发票状态']=='有效发票']
valid_bill_income = valid_bill_income.groupby(['企业代号','开票日期'])['发票状态'].count().reset_index()
#无效发票数
invalid_bill_income = bill_income[bill_income['发票状态']=='作废发票']
invalid_bill_income = invalid_bill_income.groupby(['企业代号','开票日期'])['发票状态'].count().reset_index()
#合并有效发票数，无效发票数
bill_income = pd.merge(valid_bill_income,invalid_bill_income,on=['企业代号','开票日期'],how='left')
bill_income.rename(columns={'发票状态_x':'进项有效发票数',
                            '发票状态_y':'进项作废发票数'
                            },inplace=True)
bill_income.head()

,企业代号,开票日期,进项有效发票数,进项作废发票数
0,E1,2017-07,4,NaN
1,E1,2017-08,58,3.0
2,E1,2017-09,96,2.0
3,E1,2017-10,60,5.0
4,E1,2017-11,82,1.0


In [16]:
#计算每个企业销项，作废发票数，有效发票数
bill_outcome = df1_3[['企业代号','价税合计','发票状态','开票日期']]

#有效发票数
valid_bill_outcome = bill_outcome[bill_outcome['发票状态']=='有效发票']
valid_bill_outcome = valid_bill_outcome.groupby(['企业代号','开票日期'])['发票状态'].count().reset_index()
#无效发票数
invalid_bill_outcome = bill_outcome[bill_outcome['发票状态']=='作废发票']
invalid_bill_outcome = invalid_bill_outcome.groupby(['企业代号','开票日期'])['发票状态'].count().reset_index()
#合并有效发票数，无效发票数
bill_outcome = pd.merge(valid_bill_outcome,invalid_bill_outcome,on=['企业代号','开票日期'],how='left')
bill_outcome.rename(columns={'发票状态_x':'销项有效发票数',
                            '发票状态_y':'销项作废发票数'
                            },inplace=True)
bill_outcome.head()

,企业代号,开票日期,销项有效发票数,销项作废发票数
0,E1,2017-08,280,9.0
1,E1,2017-09,346,11.0
2,E1,2017-10,291,10.0
3,E1,2017-11,299,3.0
4,E1,2017-12,194,4.0


In [40]:
'''
字段：企业代号，企业名称，信誉评级，是否违约，进项合计，销项合计，进项有效发票数，销项有效发票数
进项作废发票数，销项作废发票数，成立时间，客户量
'''
import math
data = pd.merge(df1_1,live_time,on='企业代号')
data = pd.merge(data,up_client,on='企业代号',how='left')
data = pd.merge(data,down_client,on=['企业代号','开票日期'],how='left')
data = pd.merge(data,bill_income,on=['企业代号','开票日期'],how='left')
data = pd.merge(data,bill_outcome,on=['企业代号','开票日期'],how='left')
data = pd.merge(data,out_in_mean,on=['企业代号','开票日期'],how='left')
data = pd.merge(data,out_in_std,on=['企业代号','开票日期'],how='left')
data = pd.merge(data,out_in_sum,on=['企业代号','开票日期'],how='left')
data['利润'] = data['销项税合计'] - data['进项税合计']
data['生存年限'] = data['生存年限'].map(lambda x: math.ceil(int(x)))
data['收益率'] = (data['销项税合计'] - data['进项税合计'])/data['进项税合计']
data['收益率'] = data['收益率'] /data['生存年限']
#data.head()

In [50]:
data['进项异变系数'] = data['进项每月金额标准差'] / data['进项每月均值']
data1 = pd.DataFrame(data.groupby('企业代号')['进项异变系数'].mean())
data1.to_excel('进项异变系数.xlsx')

In [41]:
data.to_excel('res123.xlsx')

In [ ]:
#提取三个指标
year_client_rate = data[['生存年限','客户数','收益率']]
year_client_rate.head()

,生存年限,客户数,收益率
0,3,352,0.145444
1,4,1579,-0.188198
2,4,135,-0.228937
3,4,26,-0.209456
4,3,33,-0.014743


In [ ]:
#对企业进行聚类
from sklearn.cluster import KMeans
kms = KMeans(n_clusters=4)
kms.fit(year_client_rate)
label = kms.labels_
label = pd.DataFrame(list(label))

data['分类'] = label
data.to_excel('data_kmeans.xlsx')
data.head()

,企业代号,企业名称,信誉评级,是否违约,生存年限,客户数,进项有效发票数,进项作废发票数,销项有效发票数,销项作废发票数,销项税合计,进项税合计,利润,收益率,分类
0,E1,***电器销售有限公司,A,否,3,352,3249,192.0,7886,224.0,6.892694e+09,4.798812e+09,2.093882e+09,0.145444,0
1,E2,***技术有限责任公司,A,否,4,1579,31435,721.0,11665,1042.0,1.717365e+08,6.947082e+08,-5.229717e+08,-0.188198,3
2,E3,***电子(中国)有限公司***分公司,C,否,4,135,4367,194.0,23688,385.0,5.752084e+07,6.827398e+08,-6.252189e+08,-0.228937,0
3,E4,***发展有限责任公司,C,否,4,26,521,37.0,2041,190.0,3.804074e+08,2.345631e+09,-1.965223e+09,-0.209456,0
4,E5,***供应链管理有限公司,B,否,3,33,2084,85.0,1005,55.0,2.320502e+08,2.427882e+08,-1.073796e+07,-0.014743,0


In [ ]:
#查看聚类后每类特征
kind_0 = data[data['分类']==0]
kind_0[['生存年限','客户数','收益率']].describe()
kind_1 = data[data['分类']==1]
kind_2 = data[data['分类']==2]
kind_3 = data[data['分类']==3]

In [ ]:
!pip install pyecharts

     |████████████████████████████████| 143kB 4.4MB/s 
     |████████████████████████████████| 133kB 6.3MB/s 


In [ ]:
from pyecharts import options as opts
from pyecharts.globals import ThemeType
from pyecharts.charts import Radar

value_0 = [list(kind_0.values)]
value_1 = [list(kind_1.values)]
value_2 = [list(kind_2.values)]
value_3 = [list(kind_3.values)]
c_schema = [
            {'name':"成立年限",'max':5,'min':0},
            {'name':"客户数",'max':8500,'min':0},
            {'name':"收益率",'max':5000,'min':0},
]
c = (
    Radar(init_opts=opts.InitOpts(theme=ThemeType.DARK))
    .add_schema(schema=c_schema,shape='circle')
    .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
    .set_series_opts(title_opts=opts.TitleOpts(title='企业聚类结果'))
)
c.render('res.html')


'/content/res.html'